# Desafío 2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import multiprocessing
from gensim.models import Word2Vec

Se nos pide elegir un tema de interés de donde vamos a generar otro dataset para usar Gensim.

En este caso y dado el background personal en energía eólica, elegí un libro muy conocido y referente en el tema, *Wind Energy Handbook* de Burton.

Para ello primero tenemos que cargar el pdf y pasarlo a una variable de python.

In [ ]:
# Usamos la libreria pymupdf
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 60.5 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF se importa como 'fitz'

# Ruta del archivo PDF
pdf_path = 'Wind_Energy_Handbook.pdf'

# Abre el archivo PDF
doc = fitz.open(pdf_path)

# Extrae el texto de todas las páginas
text = ""
for page_num in range(len(doc)):
    page = doc.load_page(page_num)  # Carga una página
    text += page.get_text()  # Extrae el texto de la página

doc.close()

# Muestra el texto extraído
print(text)

WIND ENERGY
HANDBOOK
WIND ENERGY
HANDBOOK
Tony Burton
Wind Energy Consultant, Carno, UK
David Sharpe
CREST, Loughborough University, UK
Nick Jenkins
UMIST, Manchester, UK
Ervin Bossanyi
Garrad Hassan & Partners, Bristol, UK
JOHN WILEY & SONS, LTD
Chichester • New York • Weinheim • Brisbane • Singapore • Toronto
Copyright # 2001 by John Wiley & Sons, Ltd
Bafﬁns Lane, Chichester
West Sussex, PO19 1UD, England
National
01243 779777
International (þ44) 1243 779777
e-mail (for orders and customer service enquiries): cs-books@wiley.co.uk
Visit our Home Page on: http://www.wiley.co.uk or http://www.wiley.com
All Rights Reserved. No part of this publication may be reproduced, stored in a retrieval system, or
transmitted, in any form or by any means, electronic, mechanical, photocopying, recording, scanning or
otherwise, except under the terms of the Copyright Designs and Patents Act 1988 or under the terms of a
licence issued by the Copyright Licensing Agency, 90 Tottenham Court Road, London, 

Luego de tener el texto del libro, pasamos a generar la tokenización con tensorflow.

In [ ]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
sentence_tokens.append(text_to_word_sequence(text))

In [ ]:
# Convertir la lista a un arreglo de NumPy
sentence_tokens_np = np.array(sentence_tokens)

# Cuantos tokens tiene este dataset?
print(sentence_tokens_np.shape)

(1, 241952)


Hay +200.000 tokens diferentes para este vocabulario, sin embargo, dado las características del libro, seguramente hayan muchos de ellos que se mencionen muy pocas veces o sólo una única vez y que por lo tanto no aporten realmente al corpus.

Para eso generamos el modelo word2vec de gensim e imponemos los parametros.

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crearmos el modelo generador de vectores
w2v_model = Word2Vec(min_count=4,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=3,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples
                     workers=1,
                     sg=0)           # modelo 0:CBOW  1:skipgram

In [ ]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de palabras distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 4581


La cantidad de palabras final es mucho más baja de la que se esperaba en un principio, y seguramente sea a partir de dos razones:

* La construcción con la librería usada no logro identificar correctamente todo el vocabulario y quizás no todas las palabras están bien separadas.
* El tema del libro, al ser un libro de tecnología no hay tantas palabras que se repitan un poco (minimo 4 veces) y seguramente hayan algunas como "wind", "theory" o "energy" que se repitan mucho más veces.

Entrenemos el embedding ahora con este corpus:

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 117299.375
Loss after epoch 1: 57570.3125
Loss after epoch 2: 45043.03125
Loss after epoch 3: 41562.09375
Loss after epoch 4: 39930.8125
Loss after epoch 5: 39089.8125
Loss after epoch 6: 38329.5625
Loss after epoch 7: 37708.5
Loss after epoch 8: 37638.1875
Loss after epoch 9: 37217.9375
Loss after epoch 10: 36740.6875
Loss after epoch 11: 36385.8125
Loss after epoch 12: 35979.125
Loss after epoch 13: 35727.125
Loss after epoch 14: 35325.1875
Loss after epoch 15: 34983.6875
Loss after epoch 16: 34572.25
Loss after epoch 17: 34279.25
Loss after epoch 18: 33833.1875
Loss after epoch 19: 33481.375
Loss after epoch 20: 33145.75
Loss after epoch 21: 32550.3125
Loss after epoch 22: 32200.6875
Loss after epoch 23: 31747.875
Loss after epoch 24: 31306.5
Loss after epoch 25: 30660.375
Loss after epoch 26: 29449.0625
Loss after epoch 27: 28463.625
Loss after epoch 28: 27864.125
Loss after epoch 29: 27483.25
Loss after epoch 30: 26982.75
Loss after epoch 31: 26513.125
Loss aft

(1000000, 24195200)

Y ahora lo más entretenido, buscaremos 5 palabras para identificar cuales son las más similares según el modelo. Elegí este tema de donde tengo conocimiento previo para poder entender, o no, la similitud que encuentra el modelo.

Las palabras que me interesa ver son:



1.   Wind
2.   Energy
3.   Turbines
4.   Blade
5.   Theory



In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["wind"], topn=10)

[('particular', 0.5824222564697266),
 ('located', 0.5815315246582031),
 ('rated', 0.5794485807418823),
 ('emi', 0.5672159194946289),
 ('projects', 0.5626503825187683),
 ('economic', 0.5623258948326111),
 ('higher', 0.5616702437400818),
 ('ur', 0.5560944676399231),
 ('annual', 0.5462943911552429),
 ('technology', 0.5445113182067871)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["energy"], topn=10)

[('mechanisms', 0.7276281118392944),
 ('european', 0.7008916735649109),
 ('permission', 0.6642263531684875),
 ('union', 0.656160295009613),
 ('future', 0.6545231938362122),
 ('renewable', 0.6520335674285889),
 ('et', 0.633574366569519),
 ('support', 0.6293456554412842),
 ('effective', 0.624286413192749),
 ('generation', 0.6178922653198242)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["turbines"], topn=10)

[('farms', 0.7595711946487427),
 ('farm', 0.7502105236053467),
 ('installation', 0.6344320774078369),
 ('de', 0.6041749119758606),
 ('offshore', 0.5928000211715698),
 ('349', 0.5847274661064148),
 ('amounts', 0.5821133255958557),
 ('windmills', 0.5761721134185791),
 ('machines', 0.5746153593063354),
 ('horizontal', 0.5741556286811829)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["blade"], topn=10)

[('loss', 0.6685230135917664),
 ('radius', 0.6559450626373291),
 ('407', 0.629433274269104),
 ('r', 0.6010537147521973),
 ('tip', 0.5979846715927124),
 ('element–momentum', 0.5908672213554382),
 ('stationary', 0.5865008234977722),
 ('ﬁxings', 0.582336962223053),
 ('intermediate', 0.5801212787628174),
 ('417', 0.5795536637306213)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["theory"], topn=10)

[('59', 0.8293458819389343),
 ('momentum', 0.8267479538917542),
 ('bem', 0.8075509667396545),
 ('52', 0.793022096157074),
 ('46', 0.7927283048629761),
 ('61', 0.7891860008239746),
 ('cylinder', 0.7825965881347656),
 ('actuator', 0.775688111782074),
 ('47', 0.7746150493621826),
 ('126', 0.7577939033508301)]

Veamos uno a uno como se desempeño el modelo de embeddings:

* **Wind:** No parece tener mucha similitud con el resto de palabras, parece que se construyo teniendo en cuenta a wind como proyectos de energía eólica (located, rated, particular, annual, projects) y sus beneficios (economic, higher, technology).

* **Energy:** Aquí si se ven dos contextos que encuentra para la palabra energy, los relacionados a la tecnología del futuro y renovable (mechanism, renewable, future, effective, generation), y por otro lado a temas de permisos y promociones estatales (european, permission, union, support).

* **Turbines:** Acá encuentra similitud con parques eólicos y varias turbinas juntas.

* **Blade:** Para blade encuentra conceptos más bien relacionados con la física de un perfil aerodinamico, tanto elementos que describen al elemento (loss, tip, r, radius) como los que describen al viento con el que interacciona (stationary, element-momentum)

* **Theory:** Este sin dudas es el más decepcionante, se entrena equivocadamente con mayoría de números, pensando en el contexto los números deben de hacer referencia a una ecuación y el modelo le da mucha más importancia de la que debería de tener. Hay un número en particular que es interesante que es el 59 con el que tiene más similitud, 0.59 es el conocido como límite de Betz que hace referencia a la cantidad de energía disponible del viento para generación y es un límite teórico, es interesante pensar si esta relación es por esa razón o es casualidad.

En conclusión es interesante ver como encuentra algunas similitudes, pero que le cuesta y se distrae con algunas "palabras" en el corpus, cómo es el caso de theory.

Grafiquemos para ver si encontramos alguna conclusión más de como funciona el modelo.

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

De la gráfica podemos ver algunas conclusiones interesantes:



*   Los números tienden a agruparse mucho entre sí.
*   Las palabras que son conectores (however, than, then, there, that) y no tienen que ver especificamente con este tema se agrupan por separado.
*   Voltage, Power, Electrical, se encuentran muy cerca lo cual tiene mucho sentido.
*   Otro grupo interesante es como algunas variables de un solo caracter, por ejemplo r hace referencia al radio, se juntan entre sí y con coefficient y function.

En general se pueden sacar algunas ideas de los grupos que se arman, con las limitantes que se generan de usar un PCA. En cada caso de estos grupos, siempre hay alguna palabra que no parece tener mucho sentido que se agrupe con las otras o hay otra palabra que uno esperaría ahí y el modelo la designa mucho más lejos.

Creo que este es un muy buen aproach al uso de gensim y generar un corpus con temas de mi interés, pero creo que al tener un corpus bastante pequeño y que muchas de estas "palabras" sean números genera un gran limitante en las capacidades de este modelo.

